In [1]:
import pandas as pd
import requests
from rich.progress import track
import ast
import numpy as np



In [2]:
df = pd.read_csv('swarfarm_spells.csv')

In [3]:
df

,name,description,slot,cooltime,hits,passive,aoe,random,multiplier_formula,multiplier_formula_raw,...,upgrade_4_effect,upgrade_4_amount,upgrade_5_effect,upgrade_5_amount,upgrade_6_effect,upgrade_6_amount,upgrade_7_effect,upgrade_7_amount,upgrade_8_effect,upgrade_8_amount
0,Forbidden Ingredient (Passive),Increases the Skill Level of the monster by 1 ...,1,0.0,0,True,False,False,NaN,[],...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,Special Ingredient (Passive),"The XP granted when used as Power-up material,...",1,0.0,0,True,False,False,NaN,[],...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,Best Ingredient (Passive),The XP granted will additionally increase by 5...,2,0.0,0,True,False,False,NaN,[],...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,Splinter Attack,Hurls sharp pieces of lumber and stuns the ene...,1,0.0,1,False,False,False,3.6*{ATK},"[[""ATK"", ""*"", 3.6]]",...,Damage +{0}%,10,Effect Rate +{0}%,10,Damage +{0}%,10,Effect Rate +{0}%,15,-1,-1
4,Splinter Attack,Hurls sharp pieces of lumber and stuns the ene...,1,0.0,1,False,False,False,3.6*{ATK},"[[""ATK"", ""*"", 3.6]]",...,Damage +{0}%,10,Effect Rate +{0}%,10,Damage +{0}%,10,Effect Rate +{0}%,15,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3795,Pulverize,Strikes the enemy with a hammer and prevents t...,1,0.0,1,False,False,False,2.4*{ATK} + 2.8*{DEF},"[[""ATK"", ""*"", 2.4], [""+""], [""DEF"", ""*"", 2.8]]",...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3796,Spinning Smash,"Swings the hammer, attacking all enemies to in...",2,3.0,1,False,True,False,3.7*{DEF},"[[""DEF"", ""*"", 3.7]]",...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3797,Powerful Smash,"Slams the enemy with a hammer, decreasing the ...",2,3.0,1,False,False,False,3.7*{ATK} + 5.0*{DEF},"[[""ATK"", ""*"", 3.7], [""+""], [""DEF"", ""*"", 5.0]]",...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3798,Spinning Smash,"Swings the hammer, attacking all enemies to in...",2,3.0,1,False,True,False,3.7*{DEF},"[[""DEF"", ""*"", 3.7]]",...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [4]:
liste = df['slot'].unique().tolist()
liste.append('*')
liste

[1, 2, 3, 4, -1, '*']

In [5]:
from types import NoneType


def extraire_variables_imbriquees(df, colonne, rename=None):
    
    df[colonne] = [ast.literal_eval(str(item)) for index, item in df[colonne].items()]
    
    df = pd.concat([df.drop([colonne], axis=1), df[colonne].apply(pd.Series)], axis=1)


    return df

def changement_effect(x, number):
    if x != 0 and x!= -1 and x != "-1":
        effect = {key + number: x[key] for key in x} # modification key
    else:
        effect = 0
    return effect

def changement_lead(x):
    if x != 0 and x!= -1 and x != "-1" and not isinstance(x, NoneType):
        effect = {key + '_leader': x[key] for key in x} # modification key
    else:
        effect = 0
    return effect

def swarfarm_monstres():
        # database swarfarm
    url = "https://swarfarm.com/api/v2/monsters/?page=1"
    r = requests.get(url=url)
    data = r.json()
    df = pd.DataFrame(data)

    for i in track(range(2,27), description="Chargement de la Database Swarfarm..."):
        url = f"https://swarfarm.com/api/v2/monsters/?page={i}"
        r = requests.get(url=url)
        data = r.json()
        df2 = pd.DataFrame(data)
        df = pd.concat([df, df2])
        
    
    # on extrait les variables du dict dans la colonne ['results'] et on supprime ce qui m'intéresse pas
    df = extraire_variables_imbriquees(df, 'results')
    df.drop(['next', 'previous'], axis=1, inplace=True)
    # On garde ce qui nous intéresse
    df_mobs_swarfarm = df[['id', 'url', 'com2us_id', 'family_id', 'name', 'element', 'archetype', 'skills', 'leader_skill', 'image_filename', 'natural_stars', 'awaken_level']]
    
    df_mobs_swarfarm['leader_skill'] = df_mobs_swarfarm['leader_skill'].apply(changement_lead)
    
    df_mobs_swarfarm = extraire_variables_imbriquees(df_mobs_swarfarm, 'leader_skill')
    
    df_mobs_swarfarm.drop([0], axis=1, inplace=True)
    
    df_mobs_swarfarm.fillna({'url_leader' : '-1',
                             'attribute_leader' : 'Aucun',
                             'amount_leader' : '-1',
                             'area_leader' : 'Aucun',
                             'id_leader' : '-1',
                             'element_leader' : '-1'}, inplace=True)
    
    # on rajoute le type des monstres dans la colonne 'name' pour les monstres non-awaken
    
    df_mobs_swarfarm.reset_index(drop=True, inplace=True)
    df_mobs_swarfarm.loc[df_mobs_swarfarm['awaken_level'] == 0, 'name'] = df_mobs_swarfarm['name'] + ' ' + df_mobs_swarfarm['element']

    
    df_mobs_swarfarm.to_excel('swarfarm_monstres.xlsx', index=False)
    df_mobs_swarfarm.to_csv('swarfarm_monstres.csv', index=False)
    
    return df_mobs_swarfarm
    

In [6]:
df_mob = swarfarm_monstres()

Output()

C:\Users\kevin\AppData\Local\Temp\ipykernel_47044\2310402933.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mobs_swarfarm['leader_skill'] = df_mobs_swarfarm['leader_skill'].apply(changement_lead)
C:\Users\kevin\AppData\Local\Temp\ipykernel_47044\2310402933.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[colonne] = [ast.literal_eval(str(item)) for index, item in df[colonne].items()]


In [7]:
df_mob

,id,url,com2us_id,family_id,name,element,archetype,skills,image_filename,natural_stars,awaken_level,id_leader,url_leader,attribute_leader,amount_leader,area_leader,element_leader
0,1,https://swarfarm.com/api/v2/monsters/1/,12302,12300,Forest Keeper Fire,Fire,Defense,[4],unit_icon_0010_2_1.png,1,0,-1,-1,Aucun,-1,Aucun,-1
1,2,https://swarfarm.com/api/v2/monsters/2/,12702,12700,Ghost Fire,Fire,Support,[8],unit_icon_0010_1_4.png,1,0,-1,-1,Aucun,-1,Aucun,-1
2,3,https://swarfarm.com/api/v2/monsters/3/,13002,13000,Horned Frog Fire,Fire,HP,[13],unit_icon_0012_1_2.png,1,0,-1,-1,Aucun,-1,Aucun,-1
3,4,https://swarfarm.com/api/v2/monsters/4/,12902,12900,Mimick Fire,Fire,Defense,[29],unit_icon_0012_1_1.png,1,0,-1,-1,Aucun,-1,Aucun,-1
4,5,https://swarfarm.com/api/v2/monsters/5/,12402,12400,Mushroom Fire,Fire,Attack,[32],unit_icon_0010_0_2.png,1,0,-1,-1,Aucun,-1,Aucun,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2548,2564,https://swarfarm.com/api/v2/monsters/2564/,121511,100000,2차각성 하르퓨 수정_오른쪽(물) Water,Water,none,[4491],unit_icon_0009_0_2.png,1,0,-1,-1,Aucun,-1,Aucun,-1
2549,2565,https://swarfarm.com/api/v2/monsters/2565/,121512,100000,2차각성 하르퓨 수정_오른쪽(불) Fire,Fire,none,[4492],unit_icon_0009_1_2.png,1,0,-1,-1,Aucun,-1,Aucun,-1
2550,2566,https://swarfarm.com/api/v2/monsters/2566/,121513,100000,2차각성 하르퓨 수정_오른쪽(바람) Wind,Wind,none,[4493],unit_icon_0009_2_2.png,1,0,-1,-1,Aucun,-1,Aucun,-1
2551,2567,https://swarfarm.com/api/v2/monsters/2567/,121514,100000,2차각성 하르퓨 수정_오른쪽(빛) Light,Light,none,[4494],unit_icon_0009_3_2.png,1,0,-1,-1,Aucun,-1,Aucun,-1


In [41]:
def spells_monstres():
        # database swarfarm
    url = "https://swarfarm.com/api/v2/skills/?page=1"
    r = requests.get(url=url)
    data = r.json()
    df = pd.DataFrame(data)

    for i in track(range(2,46), description="Chargement de la Database Spells Swarfarm..."):
        url = f"https://swarfarm.com/api/v2/skills/?page={i}"
        r = requests.get(url=url)
        data = r.json()
        df2 = pd.DataFrame(data)
        df = pd.concat([df, df2])
    return df    


In [124]:
df = spells_monstres()

Output()

In [125]:
 # on extrait les variables du dict dans la colonne ['results'] et on supprime ce qui m'intéresse pas
df = extraire_variables_imbriquees(df, 'results')
df.drop(['next', 'previous'], axis=1, inplace=True)
# On garde ce qui nous intéresse
    
df_spells = df[['id', 'description', 'slot', 'cooltime', 'hits', 'aoe', 'passive', 'effects', 'used_on', 'upgrades', 'multiplier_formula', 'scales_with']]
    
# 0 pour les valeurs nulles    
df_spells.fillna(0, inplace=True)
    
# on extrait les variables dans "effects"
df_spells = extraire_variables_imbriquees(df, 'effects')
    
# on set sur l'id du spell
df_spells.set_index('id', inplace=True)
    
# on remplace les valeurs nulles du effect par 0
df_spells.fillna(0, inplace=True)
    
# on rename les colonnes des effects
df_spells.rename(columns={0 : 'Effet1', 1 : 'Effet2', 2: 'Effet3', 3: 'Effet4', 4: 'Effet5', 5: 'Effet6'}, inplace=True)
    

    

            
# on modifie les colonnes effects pour plus de lisibilité
df_spells['Effet1'] = df_spells['Effet1'].apply(changement_effect, number="_1")
df_spells['Effet2'] = df_spells['Effet2'].apply(changement_effect, number="_2")
df_spells['Effet3'] = df_spells['Effet3'].apply(changement_effect, number="_3")
df_spells['Effet4'] = df_spells['Effet4'].apply(changement_effect, number="_4")
df_spells['Effet5'] = df_spells['Effet5'].apply(changement_effect, number="_5")
df_spells['Effet6'] = df_spells['Effet6'].apply(changement_effect, number="_6")

    
    
# on peut désormais extraire les variables des effects
for effect in ['Effet1', 'Effet2', 'Effet3', 'Effet4', 'Effet5', 'Effet6']:
    df_spells = extraire_variables_imbriquees(df_spells, effect)
        
   
df_spells.drop([0], axis=1, inplace=True) 
       
    # si vide, pas d'effet, donc false
for num_effet in range(1,7):
    num_effet = str(num_effet)
    col_list = ['effect_' + num_effet, 'aoe_' + num_effet, 'single_target_' + num_effet,
            'self_effect_' + num_effet, 'chance_' + num_effet, 'on_crit_' + num_effet, 'on_death_' + num_effet, 'random_' + num_effet, 'quantity_' + num_effet, 'all_' + num_effet,
            'self_hp_' + num_effet,'target_hp_' + num_effet, 'damage_' + num_effet, 'note_' + num_effet]
        
    for col in col_list:
        df_spells[col].fillna(-1, inplace=True)

C:\Users\kevin\AppData\Local\Temp\ipykernel_47044\631346636.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_spells.fillna(0, inplace=True)
C:\Users\kevin\AppData\Local\Temp\ipykernel_47044\2310402933.py:8: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df = pd.concat([df.drop([colonne], axis=1), df[colonne].apply(pd.Series)], axis=1)


In [126]:
    # on refait pareil pour l'effect
df_spells['effect_1'] = df_spells['effect_1'].apply(changement_effect, number="_1")
df_spells['effect_2'] = df_spells['effect_2'].apply(changement_effect, number="_2")
df_spells['effect_3'] = df_spells['effect_3'].apply(changement_effect, number="_3")
df_spells['effect_4'] = df_spells['effect_4'].apply(changement_effect, number="_4")
df_spells['effect_5'] = df_spells['effect_5'].apply(changement_effect, number="_5")
df_spells['effect_6'] = df_spells['effect_6'].apply(changement_effect, number="_6")

In [127]:
for effect in ['effect_1', 'effect_2', 'effect_3', 'effect_4', 'effect_5', 'effect_6']:
    df_spells = extraire_variables_imbriquees(df_spells, effect)
    
df_spells.drop([0], axis=1, inplace=True) 

In [128]:
# si vide, pas d'effet, donc false
for num_effet in range(1,7):
    num_effet = str(num_effet)
    col_list = ['id_' + num_effet, 'url_' + num_effet, 'name_' + num_effet,
            'is_buff_' + num_effet, 'type_' + num_effet]
    
    col_delete = ['description_' + num_effet, 'icon_filename_' + num_effet]
    
    df_spells.drop(col_delete, axis=1, inplace=True)
        
    for col in col_list:
        df_spells[col].fillna("-1", inplace=True)
        

# Cleanage
 
df_spells.drop(['count', 'com2us_id' , 'max_level'], axis=1, inplace=True)



In [130]:
df_spells = extraire_variables_imbriquees(df_spells, 'upgrades')

df_spells.rename(columns={0 : 'upgrade_1', 1 : 'upgrade_2', 2: 'upgrade_3', 3: 'upgrade_4', 4: 'upgrade_5', 5 : 'upgrade_6', 6: 'upgrade_7', 7: 'upgrade_8'}, inplace=True)

def changement_upgrade(x):
    if x != 0 and x!= -1 and x != "-1" and x!='0':
        effect = x['effect']
        amount = x['amount']
    else:
        effect = -1
        amount = -1


    return effect, amount


for upgrade in ['upgrade_1', 'upgrade_2', 'upgrade_3', 'upgrade_4', 'upgrade_5', 'upgrade_6', 'upgrade_7', 'upgrade_8']:
    df_spells[upgrade] = df_spells[upgrade].fillna(-1)
    df_spells[[upgrade + '_effect', upgrade + '_amount']] = df_spells[upgrade].apply(changement_upgrade).apply(pd.Series)
df_spells

C:\Users\kevin\AppData\Local\Temp\ipykernel_47044\2310402933.py:8: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df = pd.concat([df.drop([colonne], axis=1), df[colonne].apply(pd.Series)], axis=1)


-1
-1
-1
{'effect': 'Damage +{0}%', 'amount': 5}
{'effect': 'Damage +{0}%', 'amount': 5}
{'effect': 'Damage +{0}%', 'amount': 5}
{'effect': 'Damage +{0}%', 'amount': 5}
{'effect': 'Damage +{0}%', 'amount': 5}
{'effect': 'Damage +{0}%', 'amount': 5}
{'effect': 'Damage +{0}%', 'amount': 5}
{'effect': 'Damage +{0}%', 'amount': 5}
{'effect': 'Damage +{0}%', 'amount': 5}
{'effect': 'Damage +{0}%', 'amount': 5}
{'effect': 'Damage +{0}%', 'amount': 5}
-1
{'effect': 'Damage +{0}%', 'amount': 5}
{'effect': 'Damage +{0}%', 'amount': 5}
{'effect': 'Damage +{0}%', 'amount': 5}
{'effect': 'Damage +{0}%', 'amount': 5}
{'effect': 'Effect Rate +{0}%', 'amount': 5}
{'effect': 'Effect Rate +{0}%', 'amount': 5}
{'effect': 'Effect Rate +{0}%', 'amount': 5}
{'effect': 'Effect Rate +{0}%', 'amount': 5}
{'effect': 'Damage +{0}%', 'amount': 5}
{'effect': 'Damage +{0}%', 'amount': 5}
{'effect': 'Damage +{0}%', 'amount': 5}
{'effect': 'Damage +{0}%', 'amount': 5}
{'effect': 'Damage +{0}%', 'amount': 5}
{'effect

,name,description,slot,cooltime,hits,passive,aoe,random,multiplier_formula,multiplier_formula_raw,...,upgrade_4_effect,upgrade_4_amount,upgrade_5_effect,upgrade_5_amount,upgrade_6_effect,upgrade_6_amount,upgrade_7_effect,upgrade_7_amount,upgrade_8_effect,upgrade_8_amount
id,,,,,,,,,,,,,,,,,,,,,
1,Forbidden Ingredient (Passive),Increases the Skill Level of the monster by 1 ...,1,0.0,0,True,False,False,,[],...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,Special Ingredient (Passive),"The XP granted when used as Power-up material,...",1,0.0,0,True,False,False,,[],...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,Best Ingredient (Passive),The XP granted will additionally increase by 5...,2,0.0,0,True,False,False,,[],...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
4,Splinter Attack,Hurls sharp pieces of lumber and stuns the ene...,1,0.0,1,False,False,False,3.6*{ATK},"[[""ATK"", ""*"", 3.6]]",...,Damage +{0}%,10,Effect Rate +{0}%,10,Damage +{0}%,10,Effect Rate +{0}%,15,-1,-1
5,Splinter Attack,Hurls sharp pieces of lumber and stuns the ene...,1,0.0,1,False,False,False,3.6*{ATK},"[[""ATK"", ""*"", 3.6]]",...,Damage +{0}%,10,Effect Rate +{0}%,10,Damage +{0}%,10,Effect Rate +{0}%,15,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4491,Deceleration Crystal,Attacks all enemies. Decreases the Attack Bar ...,1,0.0,1,False,False,False,4.0*{ATK},"[[""ATK"", ""*"", 4.0]]",...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
4492,Assault Crystal,Attacks all enemies 2 times. The damage increa...,1,0.0,1,False,False,False,1.5*{ATK}*({Current HP %} + 1),"[[""ATK"", ""*"", 1.0], [""*""], [""ATTACK_CUR_HP_RAT...",...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
4493,Removal Crystal,"Attacks all enemies 2 times, removing their be...",1,0.0,1,False,False,False,2.0*{ATK},"[[""ATK"", ""*"", 2.0]]",...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [136]:
df_spells[['id_1', 'id_2', 'id_3', 'id_4', 'id_5', 'id_6']] = df_spells[['id_1', 'id_2', 'id_3', 'id_4', 'id_5', 'id_6']].astype(int)

In [138]:
df_effects = df_spells[['id_1', 'id_2', 'id_3', 'id_4', 'id_5', 'id_6', 'name_1', 'name_2', 'name_3', 'name_4', 'name_5', 'name_6', 'url_1', 'url_2', 'url_3', 'url_4', 'url_5', 'url_6', 'is_buff_1', 'is_buff_2', 'is_buff_3', 'is_buff_4', 'is_buff_5', 'is_buff_6', 'type_1', 'type_2', 'type_3', 'type_4', 'type_5', 'type_6']]

In [140]:
df_spells.drop(['name_1', 'name_2', 'name_3', 'name_4', 'name_5', 'name_6', 'is_buff_1', 'is_buff_2', 'is_buff_3', 'is_buff_4', 'is_buff_5', 'is_buff_6', 'type_1', 'type_2', 'type_3', 'type_4', 'type_5', 'type_6', 'url_1', 'url_2', 'url_3', 'url_4', 'url_5', 'url_6'], axis=1, inplace=True)

In [20]:
# df_spells.to_excel('swarfarm_spells.xlsx', index=False)
# df_spells.to_csv('swarfarm_spells.csv', index=False)

In [21]:
# df_mob[df_mob['id'].isin(list_mob)]

In [141]:
df_effects

,id_1,id_2,id_3,id_4,id_5,id_6,name_1,name_2,name_3,name_4,...,is_buff_3,is_buff_4,is_buff_5,is_buff_6,type_1,type_2,type_3,type_4,type_5,type_6
id,,,,,,,,,,,,,,,,,,,,,
1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
4,28,-1,-1,-1,-1,-1,Stun,-1,-1,-1,...,-1,-1,-1,-1,Debuff,-1,-1,-1,-1,-1
5,28,-1,-1,-1,-1,-1,Stun,-1,-1,-1,...,-1,-1,-1,-1,Debuff,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4491,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
4492,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
4493,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [142]:
# Fusionner les colonnes 'id'
df_id = df_effects.filter(like='id').melt(value_name='id').drop('variable', axis=1)

# Fusionner les colonnes 'name'
df_name = df_effects.filter(like='name').melt(value_name='name').drop('variable', axis=1)

# Fusionner les colonnes 'buff'
df_buff = df_effects.filter(like='buff').melt(value_name='buff').drop('variable', axis=1)

df_url = df_effects.filter(like='url').melt(value_name='url').drop('variable', axis=1)

df_type = df_effects.filter(like='type').melt(value_name='type').drop('variable', axis=1)

# Concaténer les DataFrames
df_reference = pd.concat([df_id, df_name, df_buff, df_url, df_type], axis=1)

# Supprimer les lignes avec des valeurs manquantes
df_reference = df_reference.dropna()
df_reference = df_reference.drop_duplicates()

In [143]:
df_reference.set_index('id', inplace=True)

In [144]:
df_reference

,name,buff,url,type
id,,,,
-1,-1,-1,-1,-1
28,Stun,False,https://swarfarm.com/api/v2/skill-effects/28/,Debuff
18,Glancing Hit,False,https://swarfarm.com/api/v2/skill-effects/18/,Debuff
33,Decrease ATB,False,https://swarfarm.com/api/v2/skill-effects/33/,Neutral
19,Decrease ATK,False,https://swarfarm.com/api/v2/skill-effects/19/,Debuff
...,...,...,...,...
108,Limit Harmful Effects,False,https://swarfarm.com/api/v2/skill-effects/108/,Neutral
112,Seal,True,https://swarfarm.com/api/v2/skill-effects/112/,Debuff
96,Enemy can't counterattack,False,https://swarfarm.com/api/v2/skill-effects/96/,Neutral


In [145]:
df_mob.set_index('id', inplace=True)

In [146]:
df_mob[['id_leader', 'amount_leader']] = df_mob[['id_leader', 'amount_leader']].astype(int)

df_leader = df_mob[['id_leader', 'url_leader', 'attribute_leader', 'amount_leader', 'area_leader', 'element_leader']].set_index('id_leader')
df_leader.drop_duplicates(inplace=True)
df_mob.drop(['url_leader', 'attribute_leader', 'amount_leader', 'area_leader', 'element_leader'], axis=1, inplace=True)

In [147]:
from sqlalchemy import create_engine, text, Float, BigInteger
from os import environ

# https://betterprogramming.pub/how-to-execute-plain-sql-queries-with-sqlalchemy-627a3741fdb1

DB = environ.get('API_SQL')

def init_connection():
    engine = create_engine(DB,
                           echo=False,
                           connect_args={'options': '-csearch_path={}'.format('sw')})
    return engine.connect()

conn = init_connection()

In [148]:
df_mob

,url,com2us_id,family_id,name,element,archetype,skills,image_filename,natural_stars,awaken_level,id_leader
id,,,,,,,,,,,
1,https://swarfarm.com/api/v2/monsters/1/,12302,12300,Forest Keeper Fire,Fire,Defense,[4],unit_icon_0010_2_1.png,1,0,-1
2,https://swarfarm.com/api/v2/monsters/2/,12702,12700,Ghost Fire,Fire,Support,[8],unit_icon_0010_1_4.png,1,0,-1
3,https://swarfarm.com/api/v2/monsters/3/,13002,13000,Horned Frog Fire,Fire,HP,[13],unit_icon_0012_1_2.png,1,0,-1
4,https://swarfarm.com/api/v2/monsters/4/,12902,12900,Mimick Fire,Fire,Defense,[29],unit_icon_0012_1_1.png,1,0,-1
5,https://swarfarm.com/api/v2/monsters/5/,12402,12400,Mushroom Fire,Fire,Attack,[32],unit_icon_0010_0_2.png,1,0,-1
...,...,...,...,...,...,...,...,...,...,...,...
2564,https://swarfarm.com/api/v2/monsters/2564/,121511,100000,2차각성 하르퓨 수정_오른쪽(물) Water,Water,none,[4491],unit_icon_0009_0_2.png,1,0,-1
2565,https://swarfarm.com/api/v2/monsters/2565/,121512,100000,2차각성 하르퓨 수정_오른쪽(불) Fire,Fire,none,[4492],unit_icon_0009_1_2.png,1,0,-1
2566,https://swarfarm.com/api/v2/monsters/2566/,121513,100000,2차각성 하르퓨 수정_오른쪽(바람) Wind,Wind,none,[4493],unit_icon_0009_2_2.png,1,0,-1


In [149]:
df_mob.to_sql('sw_ref_monsters', conn, if_exists='replace')

553

In [150]:
df_spells.drop(['upgrade_1', 'upgrade_2', 'upgrade_3', 'upgrade_4', 'upgrade_5', 'upgrade_6', 'upgrade_7', 'upgrade_8'], axis=1).to_sql('sw_ref_spells',
                                                                                                                                        conn,
                                                                                                                                        if_exists='replace')

227

In [151]:
df_reference.to_sql('sw_ref_effects',
                    conn,
                if_exists='replace')

104

In [152]:
df_leader.to_sql('sw_ref_leader',
                conn,
                if_exists='replace')

229

In [153]:
conn.commit()